<a href="https://colab.research.google.com/github/chloe-nguyenminh/Movie_Genre_Detector_Project/blob/main/Movie_detector_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
from tensorflow.keras import layers
from tensorflow.keras import regularizers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-5ponctdk
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-5ponctdk
  Resolved https://github.com/tensorflow/docs to commit 36b0a7de4c3b2e0e06f03c0f56588e8fd155d63c
  Preparing metadata (setup.py) ... done


In [3]:
import pathlib
import shutil
import tempfile
# logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
# shutil.rmtree(logdir, ignore_errors=True)

In [4]:
# Set up: Importing numpy, pandas, etc.
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from numpy.random import RandomState
from tensorflow.keras import datasets, layers, models, Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from tqdm import tqdm

df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/data_no_dupl_with_existing_posters.csv")


In [5]:
rm -rf 'find -type d -name .ipynb_checkpoints'

In [6]:
#Resizing
img_width = 253
img_height = 253

image_dir = '/content/drive/MyDrive/Colab_Notebooks/train'
image_dir2 = '/content/drive/MyDrive/Colab_Notebooks/test'

train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=image_dir,
    validation_split=0.2,
    subset='training',
    image_size=(img_width, img_height),
    shuffle=True,
    seed=123,
    batch_size=32,
    label_mode='int'
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=image_dir2,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(img_width, img_height),
    shuffle=True,
    batch_size=32,
)

print(train_dataset)


Found 25036 files belonging to 10 classes.
Using 20029 files for training.
Found 6909 files belonging to 10 classes.
Using 1381 files for validation.
<BatchDataset element_spec=(TensorSpec(shape=(None, 253, 253, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [7]:
class_names = train_dataset.class_names
print(class_names)

['Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family', 'Music', 'Romance', 'Thriller', 'War']


In [ ]:
# plt.figure(figsize=(10,10))
# for images, labels in train_dataset.take(1):
#   for i in range(2):
#     ax = plt.subplot(3, 3, i+1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [ ]:
# for image_batch, labels_batch in train_dataset:
#   print(image_batch.shape)
#   print(labels_batch.shape)
#   break
#   # image_batch is a batch of 32 imgs of shape 253*253*3
#   # labels_batch are corresponding labels to 32 imgs

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

94765736/94765736 [==============================] - 4s 0us/step


In [11]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = tf.keras.layers.Dense(3, activation='softmax')(x)

In [12]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 253, 253, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 259, 259, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 127, 127, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [9]:
# data_augmentation = keras.Sequential(
#   [
#     layers.RandomFlip("horizontal",
#                       input_shape=(img_height, img_width, 3)),
#     layers.RandomFlip('vertical',input_shape=(img_height, img_width, 3)),
#     layers.RandomRotation(0.5),
#     layers.RandomZoom(0.5),
#   ]
# )

In [ ]:
# lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
#   0.001,
#   decay_steps=2000,
#   decay_rate=1,
#   staircase=False)

# def get_optimizer():
#   return tf.keras.optimizers.Adam(lr_schedule)

In [ ]:
# def get_callbacks(name):
#   return [
#     tfdocs.modeling.EpochDots(),
#     tf.keras.callbacks.EarlyStopping(monitor='sparse_categorical_crossentropy', patience=200),
#     tf.keras.callbacks.TensorBoard(logdir/name),
#   ]

In [ ]:
# def compile_and_fit(model, name, optimizer=None, max_epochs=10):
#   if optimizer is None:
#     optimizer = get_optimizer()
#   model.compile(optimizer=optimizer,
#                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#                 metrics=['accuracy'])

#   model.summary()

#   history = model.fit(
#     train_dataset,
#     steps_per_epoch = 2,
#     epochs=max_epochs,
#     validation_data=test_dataset,
#     callbacks=get_callbacks(name),
#     )
#   return history

In [10]:
num_classes = len(class_names)
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  # layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  # layers.BatchNormalization(),
  # layers.MaxPooling2D(),
  # layers.Conv2D(64, 3, padding='same', activation='relu'),
  # layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Dropout(0.3),
  layers.Flatten(),
  # layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
  # layers.Dropout(0.3),
  # layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
  # layers.Dropout(0.3),
  # layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
  layers.Dense(num_classes)
])

In [20]:
model.compile(optimizer='Adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [18]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='logs'),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='effnet.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='auto',
        verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.3,
        patience=2,
        min_delta=0.001,
        mode='auto',
        verbose=1)
]

In [ ]:
epochs=10
history = model.fit(
  train_dataset,
  batch_size=32,
  epochs=epochs,
  validation_data=test_dataset,
  verbose=1,
  callbacks=[callbacks]
)

Epoch 1/10
